In [38]:
%pip install openai pandas fastapi

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta
import uuid

# Generate synthetic data for the API metrics table
num_rows = 250

# Possible values for endpoints and methods
endpoints = [
    "/auth/login", "/auth/logout", "/auth/reset", 
    "/data/fetch", "/data/upload", "/data/sync",
    "/user/profile", "/user/settings", "/user/preferences",
    "/admin/dashboard", "/admin/users", "/admin/logs",
    "/api/v1/products", "/api/v1/orders", "/api/v1/customers",
    "/search/quick", "/search/advanced",
    "/payments/process", "/payments/verify"
]

methods = ["GET", "POST", "PUT", "DELETE", "PATCH"]

# Response codes with weighted probabilities
response_codes = {
    200: 0.55,  # Success (55% probability)
    201: 0.1,   # Created
    400: 0.1,   # Bad Request
    401: 0.05,  # Unauthorized
    403: 0.05,  # Forbidden
    404: 0.05,  # Not Found
    500: 0.08,  # Internal Server Error
    503: 0.02   # Service Unavailable
}

# Generate timestamps within the last 24 hours
end_date = datetime.now()
start_date = end_date - timedelta(days=1)

# Function to generate latency with suspicious patterns
def generate_latency():
    pattern_type = random.random()
    
    if pattern_type < 0.7:  # 70% normal cases
        return random.randint(50, 500)
    elif pattern_type < 0.85:  # 15% slower cases
        return random.randint(500, 2000)
    elif pattern_type < 0.95:  # 10% very slow cases
        return random.randint(2000, 10000)
    else:  # 5% suspicious/timeout cases
        return random.randint(10000, 60000)  # Up to 1 minute

# Create the data
data = {
    "RequestID": [str(uuid.uuid4()) for _ in range(num_rows)],
    "Endpoint": [random.choice(endpoints) for _ in range(num_rows)],
    "Method": [random.choice(methods) for _ in range(num_rows)],
    "Latency_ms": [generate_latency() for _ in range(num_rows)]
}

# Create DataFrame
df = pd.DataFrame(data)

# Generate timestamps and calculate request/response times with suspicious patterns
timestamps = []
time_requests = []
time_responses = []

for _ in range(num_rows):
    # Generate base timestamp
    timestamp = start_date + timedelta(
        seconds=random.randint(0, int((end_date - start_date).total_seconds()))
    )
    
    # Calculate request and response times based on latency
    latency = df['Latency_ms'].iloc[_] / 1000  # Convert to seconds
    
    # Add some suspicious timing patterns
    if random.random() < 0.1:  # 10% chance of suspicious timing
        # Add extra delay between request and response
        extra_delay = random.randint(30, 300)  # 30 seconds to 5 minutes
        latency += extra_delay
    
    request_time = timestamp
    response_time = request_time + timedelta(seconds=latency)
    
    timestamps.append(timestamp)
    time_requests.append(request_time)
    time_responses.append(response_time)

df['Timestamp'] = timestamps
df['time_request_to_Server'] = time_requests
df['Time_response_from_server'] = time_responses

# Add response codes with correlation to latency
response_codes_list = []
for latency in df['Latency_ms']:
    if latency > 10000:  # Very slow responses
        # Higher chance of error codes for very slow responses
        codes = [500, 503, 504, 408]  # Including timeout status
        response_codes_list.append(random.choice(codes))
    elif latency > 2000:  # Slower responses
        if random.random() < 0.3:  # 30% chance of error
            response_codes_list.append(random.choice([500, 503, 408]))
        else:
            response_codes_list.append(200)
    else:  # Normal responses
        response_codes_list.append(
            random.choices(
                list(response_codes.keys()),
                weights=list(response_codes.values()),
                k=1
            )[0]
        )

df['Response_Code'] = response_codes_list

# Generate request counts with suspicious patterns
df['Request_Count'] = [
    int(random.choice([
        random.randint(1, 50),       # Low traffic
        random.randint(50, 200),     # Medium traffic
        random.randint(200, 500),    # High traffic
        random.randint(500, 1000),   # Traffic spike
        random.randint(2000, 5000)   # Suspicious spike
    ]) * random.uniform(0.9, 1.1))
    for _ in range(num_rows)
]

# Sort by timestamp
df = df.sort_values('Timestamp')

# Reorder columns to match requested format
df = df[[
    'RequestID',
    'Endpoint',
    'Method',
    'Latency_ms',
    'time_request_to_Server',
    'Time_response_from_server',
    'Response_Code',
    'Request_Count',
    'Timestamp'
]]

# Display the first few rows
print("\nFirst few rows of the generated data:")
print(df.head())

# Display rows with suspicious timing (latency > 10 seconds)
suspicious_rows = df[df['Latency_ms'] > 10000]
print("\nSuspicious timing patterns (latency > 10 seconds):")
print(suspicious_rows)

# Save the data to a CSV file
file_path = "synthetic_api_data.csv"
df.to_csv(file_path, index=False)

print(f"\nData saved to {file_path}")

# Display some basic statistics
print("\nLatency statistics:")
print(df['Latency_ms'].describe())
print("\nResponse Code distribution:")
print(df['Response_Code'].value_counts())

In [ ]:
import openai
import pandas as pd
import json

# Initialize SambaNova LLM client
client = openai.OpenAI(api_key="9cc5e443-487d-450c-913d-2d027b5ea1eb", base_url="https://api.sambanova.ai/v1")

# Load the synthetic data from the CSV file
file_path = "synthetic_api_data.csv"
data = pd.read_csv(file_path)

# Limit analysis to the specified columns
data = data[[
    'RequestID',
    'Endpoint',
    'Method',
    'Latency_ms',
    'time_request_to_Server',
    'Time_response_from_server',
    'Response_Code',
    'Request_Count',
    'Timestamp'
]]

# Convert time_request_to_Server and Time_response_from_server to numeric values
data['time_request_to_Server'] = pd.to_numeric(data['time_request_to_Server'], errors='coerce')
data['Time_response_from_server'] = pd.to_numeric(data['Time_response_from_server'], errors='coerce')

# Function to get insights from SambaNova LLM
def get_llm_insights(prompt):
    response = client.chat.completions.create(
        model="Meta-Llama-3.1-8B-Instruct",
        messages=[{"role": "system", "content": prompt}],
        temperature=0.7,
        top_p=0.9
    )
    return response.choices[0].message.content

# Function to summarize metrics for an endpoint
def summarize_metrics(group):
    avg_latency = group['Latency_ms'].mean()
    avg_request_to_server = group['time_request_to_Server'].mean()
    avg_response_from_server = group['Time_response_from_server'].mean()
    total_requests = group['Request_Count'].sum()
    response_code_counts = group['Response_Code'].value_counts().to_dict()
    methods_used = group['Method'].value_counts().to_dict()
    
    # Summarize data
    summary = (
        f"Average latency: {avg_latency:.2f} ms.\n"
        f"Average time request to server: {avg_request_to_server:.2f} ms.\n"
        f"Average time response from server: {avg_response_from_server:.2f} ms.\n"
        f"Total requests: {total_requests}.\n"
        f"Response codes: {response_code_counts}.\n"
        f"Methods used: {methods_used}."
    )
    return summary

# Generate insights for each endpoint
insights = []

for endpoint, group in data.groupby("Endpoint"):
    # Summarize the metrics
    summary = summarize_metrics(group)
    
    # Use SambaNova LLM to generate analysis and suggestions
    llm_response = get_llm_insights(
        f"You are an API analytics expert. Given the following metrics for the endpoint '{endpoint}', "
        f"provide an analysis and actionable suggestions:\n\n{summary}\n\n"
        "Respond in the format: 'Analysis: <analysis>. Suggestion: <suggestion>'."
    )
    
    # Split the response into analysis and suggestion
    analysis, suggestion = "", ""
    if "Analysis:" in llm_response and "Suggestion:" in llm_response:
        analysis = llm_response.split("Analysis:")[1].split("Suggestion:")[0].strip()
        suggestion = llm_response.split("Suggestion:")[1].strip()
    else:
        analysis = llm_response  # Fallback in case format is not strictly followed
    
    # Format the output
    insights.append({
        "endpoint": endpoint,
        "analysis": analysis,
        "suggestion": suggestion
    })

# Save insights to a JSON file
insights_file = "insights.json"
with open(insights_file, "w") as f:
    json.dump(insights, f, indent=4)

# Generate overall suggestion
general_summary = "\n".join(
    [f"Endpoint: {entry['endpoint']}\nAnalysis: {entry['analysis']}\nSuggestion: {entry['suggestion']}\n"
     for entry in insights]
)
overall_prompt = (
    f"You are an API analytics expert. Based on the following insights for multiple endpoints, "
    f"provide a single generalized suggestion:\n\n{general_summary}\n\n"
    "Respond in the format: 'Overall Suggestion: <generalized suggestion>'."
)

overall_response = get_llm_insights(overall_prompt)

# Extract the overall suggestion
overall_suggestion = overall_response.split("Overall Suggestion:")[1].strip() if "Overall Suggestion:" in overall_response else overall_response

# Save overall suggestion to a JSON file
overall_suggestions_file = "overall_suggestions.json"
with open(overall_suggestions_file, "w") as f:
    json.dump({"overall_suggestion": overall_suggestion}, f, indent=4)

print(f"Insights saved to {insights_file}")
print(f"Overall suggestion saved to {overall_suggestions_file}")


In [ ]:
import openai
import pandas as pd
import os
from datetime import datetime
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

def load_data():
    """Load and prepare the API metrics data"""
    try:
        df = pd.read_csv("data.csv")
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

def generate_business_report(df):
    """Create a prompt for generating a business report from the data"""
    # Calculate key metrics
    metrics = {
        "total_requests": len(df),
        "avg_latency": df['Latency_ms'].mean(),
        "success_rate": (df['Response_Code'].astype(str).str.startswith('2').mean() * 100),
        "total_errors": len(df[df['Response_Code'].astype(str).str.startswith(('4', '5'))]),
        "suspicious_latency": len(df[df['Latency_ms'] > 10000]),
        "busiest_endpoint": df['Endpoint'].mode().iloc[0],
        "date_range": f"{df['Timestamp'].min()} to {df['Timestamp'].max()}"
    }
    
    report_prompt = f"""
    Based on the following API metrics data, generate a greatly comprehensive business report with actionable insights:
    Make sure it propely formatted in Markdown.

    Time Period: {metrics['date_range']}
    Total Requests: {metrics['total_requests']}
    Average Latency: {metrics['avg_latency']:.2f}ms
    Success Rate: {metrics['success_rate']:.2f}%
    Total Errors: {metrics['total_errors']}
    Suspicious Response Times: {metrics['suspicious_latency']}
    Most Accessed Endpoint: {metrics['busiest_endpoint']}
    
    Firstly, list the top 10 endpoints that need attention

    Please analyze this data and provide:
    1. Executive Summary
    2. Performance Analysis
    3. Security Concerns
    4. Recommendations
    Make sure to highlight any suspicious patterns or potential issues.
    """
    return report_prompt

def process_question(df, question):
    """Prepare context for answering specific questions about the data"""
    context = f"""Based on the API metrics data, answer the following question: {question}

    Relevant metrics:
    - Total requests: {len(df)}
    - Average latency: {df['Latency_ms'].mean():.2f}ms
    - Success rate: {(df['Response_Code'].astype(str).str.startswith('2').mean() * 100):.2f}%
    - Error count: {len(df[df['Response_Code'].astype(str).str.startswith(('4', '5'))])}
    - Endpoints: {', '.join(df['Endpoint'].unique())}
    """
    return context

class APIMetricsChat:
    def __init__(self, api_key):
        self.client = api_key
        self.df = load_data()
        self.conversation_history = []

    def generate_initial_report(self):
        if self.df is None:
            return "Error: Could not load data"

        report_prompt = generate_business_report(self.df)
        
        try:
            response = self.client.chat.completions.create(
                model='Meta-Llama-3.1-8B-Instruct',
                messages=[
                    {"role": "system", "content": "You are a business analyst specializing in API metrics analysis."},
                    {"role": "user", "content": report_prompt}
                ],
                temperature=0.1,
                top_p=0.1
            )
            report = response.choices[0].message.content
            self.conversation_history.append({"role": "system", "content": report})
            return report
        except Exception as e:
            return f"Error generating report: {e}"

    def answer_question(self, question):
        if self.df is None:
            return "Error: Could not load data"

        context = process_question(self.df, question)
        
        messages = [
            {"role": "system", "content": "You are a business analyst specializing in API metrics analysis."},
            *self.conversation_history,
            {"role": "user", "content": context}
        ]

        try:
            response = self.client.chat.completions.create(
                model='Meta-Llama-3.1-8B-Instruct',
                messages=messages,
                temperature=0.1,
                top_p=0.1
            )
            answer = response.choices[0].message.content
            self.conversation_history.append({"role": "user", "content": question})
            self.conversation_history.append({"role": "assistant", "content": answer})
            return answer
        except Exception as e:
            return f"Error answering question: {e}"

# Initialize the chatbot (run this in a cell)
api_key = openai.OpenAI(api_key="9cc5e443-487d-450c-913d-2d027b5ea1eb", base_url="https://api.sambanova.ai/v1")
chatbot = APIMetricsChat(api_key)

# Generate initial report (run this in a cell)
print("Generating initial business report...\n")
report = chatbot.generate_initial_report()

# Save the report to a markdown file
# with open("business_report.md", "w") as file:
#     file.write(report)
# print(report)

print("\n" + "="*50 + "\n")
print("You can now ask questions about the API metrics data in subsequent cells using:")
print("chatbot.answer_question('your question here')")

# Ask specific questions (run this in a cell)
app = FastAPI()

class Question(BaseModel):
    question: str

@app.post("/ask")
async def ask_question(question: Question):
    if chatbot.df is None:
        raise HTTPException(status_code=500, detail="Error: Could not load data")
    
    answer = chatbot.answer_question(question.question)
    return {"answer": answer}

uvicorn.run(app, host="0.0.0.0", port=8000)

# Example usage:
# Run the FastAPI app with: uvicorn filename:app --reload
# Then you can ask questions by sending a POST request to /ask with a JSON body: {"question": "your question here"}

# Ask a question
question = input("You: ")
answer = chatbot.answer_question(question)
print(answer)

Generating initial business report...



You can now ask questions about the API metrics data in subsequent cells using:
chatbot.answer_question('your question here')
**Endpoint Analysis: /api/v1/orders**

**Total Requests:** 75
**Average Latency:** 2500.12ms
**Success Rate:** 45.33%
**Error Count:** 43

**Key Observations:**

* The /api/v1/orders endpoint received a total of 75 requests, which is a significant portion of the total requests (30%).
* The average latency of 2500.12ms is higher than the overall average latency of 2280.23ms, indicating that this endpoint is taking longer to respond to requests.
* The success rate of 45.33% is lower than the overall success rate of 51.60%, indicating that a higher percentage of requests to this endpoint are failing.
* The error count of 43 is a significant portion of the total errors (35%).

**Potential Causes:**

* The high average latency may be due to slow database queries or inefficient code in the /api/v1/orders endpoint.
* The low suc